---
title: Organizing founder haplotype probabilities into SQLite database
author: Sabrina Mi
date: 12/6/2023
---

We want to store the qtl2 outputs in a database that is easily queryable by sample ID and genomic region.

In [22]:
import os
import pandas as pd
import sqlite3
import numpy as np
probabilities_dir = '/home/s1mi/Br_genotype_probabilities'
individuals = [d for d in os.listdir(probabilities_dir) if os.path.isdir(os.path.join(probabilities_dir, d))]

In [13]:
# Function to read CSV files and create tables in SQLite database
def create_sqlite_db(N):
    # Connect to SQLite database
    conn = sqlite3.connect(os.path.join(probabilities_dir, f'chr{N}_probabilities.db'))
    
    for individual in individuals:
        check_query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{individual}';"
        # Read the result of the query into a DataFrame
        existing_table = pd.read_sql_query(check_query, conn)

        # If the table doesn't exist, create table
        if existing_table.empty:
            prob_csv = os.path.join(probabilities_dir, individual, f'chr{N}_probabilities.csv')
            df = pd.read_csv(prob_csv, index_col=0)
            positions = df.index.str.replace(f"chr{N}:", "").astype(int) 
            df.insert(0, 'POS', positions)
            df.to_sql(individual, conn, if_exists='replace', index=False)
    # Commit changes and close connection
    conn.commit()
    conn.close()


In [ ]:
for N in range(1,21):
    print("Working on chromosome", N)
    create_sqlite_db(N)


## Preview Database

In [17]:
# Connect to the SQLite database
conn = sqlite3.connect('/eagle/AIHPC4Edu/sabrina/Br_genotype_probabilities/chr20_probabilities.db')

# Create a cursor object
cursor = conn.cursor()

query_tables = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query to get table names
cursor.execute(query_tables)

# Fetch all table names
tables = cursor.fetchall()

# Iterate through tables and print their data
for table in tables[:5]:
    table_name = table[0]
    print(f"Table: {table_name}")
    
    # Query to fetch all data from the table
    query_data = f"SELECT * FROM `{table_name}` LIMIT 3;"
    cursor.execute(query_data)
    
    # Fetch all data from the table
    table_data = cursor.fetchall()
    
    # Print the table's data
    for row in table_data:
        print(row)
# Close the database connection
conn.close()


Table: 00078A02B6
(3201, 0.499262964281572, 1.88520490019066e-06, 0.162477747051664, 0.0129312367910615, 0.162477747062661, 0.162477747068122, 1.88520490019106e-06, 0.0003687873347151)
(7226, 0.499997674934029, 4.75965339555511e-08, 0.162469115322473, 0.0125925304011694, 0.162469115333482, 0.162469115338949, 4.75965339555847e-08, 2.35347571803293e-06)
(7266, 0.499999935708622, 3.16496421849708e-08, 0.162469769268664, 0.0125905455363656, 0.162469769279672, 0.162469769285139, 3.16496421849787e-08, 1.47622298115919e-07)
Table: 00078A16DB
(3201, 0.499262964281493, 1.88520490019009e-06, 0.162477747051738, 0.0129312367910557, 0.16247774706207, 0.162477747068565, 1.88520490019092e-06, 0.0003687873347151)
(7226, 0.499997674934177, 4.75965339555652e-08, 0.162469115322621, 0.0125925304011694, 0.162469115332964, 0.162469115339466, 4.75965339555987e-08, 2.35347571803365e-06)
(7266, 0.499999935708543, 3.16496421849658e-08, 0.162469769268737, 0.0125905455363599, 0.162469769279081, 0.162469769285583,

In [29]:
interval = ('chr20', 7230- 512, 7230 + 512)
probabilities = {}
for individual in individuals[:3]:
    db = os.path.join(probabilities_dir, f"{interval[0]}_probabilities.db")
    conn = sqlite3.connect(db)
    # Create a cursor object
    cursor = conn.cursor()
    # Parameterized query to select rows within the specified interval for the 'POS' column
    query = f"SELECT * FROM `{table_name}` WHERE POS BETWEEN {interval[1]} AND {interval[2]};"
    # Execute the query with the interval bounds as parameters
    df = pd.read_sql_query(query, conn)
    probabilities[individual] = df
print(probabilities)
    


{'00078A02B6':     POS       ACI            BN       BUF      F344      M520        MR  \
0  7226  0.499998  4.759653e-08  0.162469  0.012593  0.162469  0.162469   
1  7266  0.500000  3.164964e-08  0.162470  0.012591  0.162470  0.162470   
2  7269  0.500000  3.039399e-08  0.162470  0.012590  0.162470  0.162470   
3  7341  0.500000  2.313504e-09  0.162471  0.012587  0.162471  0.162471   
4  7581  0.500000  2.433167e-09  0.162475  0.012575  0.162475  0.162475   

            WKY            WN  
0  4.759653e-08  2.353476e-06  
1  3.164964e-08  1.476223e-07  
2  3.039399e-08  1.459054e-07  
3  2.313504e-09  1.265991e-07  
4  2.433167e-09  6.040197e-08  , '00078A16DB':     POS       ACI            BN       BUF      F344      M520        MR  \
0  7226  0.499998  4.759653e-08  0.162469  0.012593  0.162469  0.162469   
1  7266  0.500000  3.164964e-08  0.162470  0.012591  0.162470  0.162470   
2  7269  0.500000  3.039399e-08  0.162470  0.012590  0.162470  0.162470   
3  7341  0.500000  2.313504